## knock 074 月曜日からの経過日数を計算する

In [8]:
import polars as pl
pl.Config.set_tbl_cols(-1)# 列の表示が省略されないようにする
import polars.selectors as cs# 抽出条件のプリセット

### データを読み込む

In [9]:
df_receipt = pl.read_csv("../docker/work/data/receipt.csv")
display(df_receipt.head())

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
i64,i64,str,i64,i64,str,str,i64,i64
20181103,1541203200,"""S14006""",112,1,"""CS006214000001…","""P070305012""",1,158
20181118,1542499200,"""S13008""",1132,2,"""CS008415000097…","""P070701017""",1,81
20170712,1499817600,"""S14028""",1102,1,"""CS028414000014…","""P060101005""",1,170
20190205,1549324800,"""S14042""",1132,1,"""ZZ000000000000…","""P050301001""",1,25
20180821,1534809600,"""S14025""",1102,2,"""CS025415000050…","""P060102007""",1,90


### ノック

In [10]:
(
    df_receipt
    # 使用する列を抽出する(大規模データの場合メモリ使用量を節約する効果がある)
    .select(
        pl.col("sales_ymd")
    )
    # DateTime型に変換する
    .with_columns([
        pl.col("sales_ymd").cast(pl.Utf8).str.strptime(dtype = pl.Date, format = "%Y%m%d")
    ])
    # 月曜日からの経過日数を計算する
    .with_columns([
        (pl.col('sales_ymd').dt.weekday() - 1).alias('elapsed_days'),
        pl.col('sales_ymd').dt.truncate('1w').alias('monday')
    ])
    # 先頭10行
    .head(n = 10)
)

sales_ymd,elapsed_days,monday
date,u32,date
2018-11-03,5,2018-10-29
2018-11-18,6,2018-11-12
2017-07-12,2,2017-07-10
2019-02-05,1,2019-02-04
2018-08-21,1,2018-08-20
2019-06-05,2,2019-06-03
2018-12-05,2,2018-12-03
2019-09-22,6,2019-09-16
2017-05-04,3,2017-05-01
